In [30]:
from sklearn.datasets import make_moons

records = 1000
X_moons, y_moons = make_moons(records, noise=0.1, random_state=2017)

In [34]:
import numpy as np
X_moons_bias = np.c_[np.ones((records,1)),X_moons]

In [38]:
y_moons_cv = y_moons.reshape(-1,1)

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X_moons_bias , y_moons_cv , test_size = 0.2, random_state = 2017)

In [46]:
def mini_batch(X_train, y_train, batch_size):
    random_indices = np.random.randint(0, len(X_train), batch_size)
    X_batch = X_train[random_indices]
    y_batch = y_train[random_indices]
    return X_batch, y_batch

In [49]:
import tensorflow as tf

In [52]:
tf.reset_default_graph()

In [55]:
input_features = 2

In [58]:
X = tf.placeholder(tf.float32, shape=(None, input_features + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([input_features + 1, 1], -1.0, 1.0, seed=42), name="theta")
logits = tf.matmul(X, theta, name="logits")
y_proba = tf.sigmoid(logits)

In [59]:
loss = tf.losses.log_loss(y, y_proba)

In [60]:
learning_rate = 0.01
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)

In [61]:
init = tf.global_variables_initializer()

In [66]:
n_epochs = 2000
batch_size = 100
n_batches = int(np.ceil(records / batch_size))  #20

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = mini_batch(X_train, y_train, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        loss_val = loss.eval({X: X_test, y: y_test})
        if epoch % 100 == 0:
            print("Epoch:", epoch, "\tLoss:", loss_val)

    y_probability = y_proba.eval(feed_dict={X: X_test, y: y_test})

Epoch: 0 	Loss: 0.912207
Epoch: 100 	Loss: 0.412361
Epoch: 200 	Loss: 0.352756
Epoch: 300 	Loss: 0.329289
Epoch: 400 	Loss: 0.315743
Epoch: 500 	Loss: 0.30658
Epoch: 600 	Loss: 0.300046
Epoch: 700 	Loss: 0.29527
Epoch: 800 	Loss: 0.291423
Epoch: 900 	Loss: 0.288524
Epoch: 1000 	Loss: 0.28624
Epoch: 1100 	Loss: 0.284496
Epoch: 1200 	Loss: 0.283202
Epoch: 1300 	Loss: 0.282134
Epoch: 1400 	Loss: 0.28118
Epoch: 1500 	Loss: 0.280488
Epoch: 1600 	Loss: 0.279916
Epoch: 1700 	Loss: 0.279365
Epoch: 1800 	Loss: 0.27896
Epoch: 1900 	Loss: 0.278605


In [69]:
y_predicted = (y_probability >= 0.5)

In [72]:
from sklearn.metrics import precision_score, recall_score
print("Precision: {:.3f}".format(precision_score(y_test, y_predicted)))
print("Recall: {:.3f}".format(recall_score(y_test, y_predicted)))

Precision: 0.883
Recall: 0.865
